In [1]:
#Importing libraries for scraping

import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

#Importing Selenium web driver
from selenium import webdriver

#Importing exceptions which needed to be handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException,ElementClickInterceptedException

#Importing request
import requests

#import nltk
import nltk
from nltk.tokenize import regexp_tokenize
#Importing regex
import re

# Create a soup object
soup = BeautifulSoup()

from urllib.parse import urljoin
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [32]:
#Activating the Chrome Browser
driver=  webdriver.Chrome("chromedriver.exe")
time.sleep(2)
driver.maximize_window()

In [5]:
#Empty Lists
Airline=[]
Date_of_Journey=[]
Source=[]
Destination=[]
Dep_time=[]
Arrival_Time=[]
Duration=[]
Total_Stops=[]
Price=[]

In [42]:
url = 'https://flight.easemytrip.com/FlightList/Index?srch=MAA-Chennai-India|COK-Cochin(Kochi)-India|25/10/2021&px=1-0-0&cbn=0&ar=undefined&isow=true&isdm=true&lng=&CouponCode=&ompAff=&bc=&ISWL='
driver.get(url)
time.sleep(3)


previous_height =driver.execute_script('return document.body.scrollHeight')

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(3)

    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == previous_height:
        break
    previous_height = new_height

time.sleep(5)

for j in range(10):
    nxt_page =driver.find_element_by_xpath('//*[@id="ServiceControllerId"]/div[2]/div/a[2]')

    time.sleep(10)

    #Airline name
    titles=driver.find_elements_by_xpath('//span[@class="txt-r4 ng-binding"]')
    for a in titles:
         Airline.append(a.text)
    time.sleep(2)
    #Source
    source= driver.find_elements_by_xpath('//*[@id="ResultDiv"]/div/div/div[4]/div/div[1]/div[1]/div[2]/span[2]')
    for sr in source:
        Source.append(sr.text)

    time.sleep(2)

    #Destination
    destiny=driver.find_elements_by_xpath('//*[@id="ResultDiv"]/div/div/div[4]/div/div[1]/div[1]/div[4]/span[2]')
    for d in destiny:
        Destination.append(d.text)

    #Departure_time
    depart=driver.find_elements_by_xpath('//*[@id="ResultDiv"]/div/div/div[4]/div/div[1]/div[1]/div[2]/span[1]')
    for de in depart:
        Dep_time.append(de.text)

    #Arrival_time
    arrival=driver.find_elements_by_xpath('//*[@id="ResultDiv"]/div/div/div[4]/div/div[1]/div[1]/div[4]/span[1]')
    for arr in arrival:
        Arrival_Time.append(arr.text)

    #Duration
    duration=driver.find_elements_by_xpath('//*[@id="ResultDiv"]/div/div/div[4]/div/div[1]/div[1]/div[3]/span[1]')
    for du in duration:
        Duration.append(du.text)


    #Total stops
    stops=driver.find_elements_by_xpath('//*[@id="ResultDiv"]/div/div/div[4]/div/div[1]/div[1]/div[3]/span[2]')
    for st in stops:
        Total_Stops.append(st.text)

    #Price
    price=driver.find_elements_by_xpath('//div[@class="col-md-8 col-sm-8 col-xs-9 txt-r6-n ng-binding"]')
    for p in price:
        Price.append(p.text)

    #Flight detail
    try:
        detail =driver.find_elements_by_xpath("//div[@class='d-up']")
        for i in detail:
            i.click()
            time.sleep(2)
            #Date of journey
            date_list= driver.find_elements_by_xpath('/html/body/form/div[9]/div[4]/div/div[2]/div[2]/div/div/div[4]/div/div[1]/div[8]/div[2]/div/div[1]/div[2]/div[3]/span[2]')
            for date in date_list:
                Date_of_Journey.append(date.text)
                time.sleep(2)

    except NoSuchElementException:
        pass

    nxt_page.click()    
    

In [45]:
#Changing the date into right format
Date_of_Journey=[value.split('-')[1] for value in Date_of_Journey]


In [46]:
Date_of_Journey=pd.to_datetime(pd.Series(Date_of_Journey), format='%d%b%Y')
Date_of_Journey

0      2021-10-24
1      2021-10-24
2      2021-10-24
3      2021-10-24
4      2021-10-24
          ...    
8664   2021-11-03
8665   2021-11-03
8666   2021-11-03
8667   2021-11-03
8668   2021-11-03
Length: 8669, dtype: datetime64[ns]

In [47]:
#selecting departure time hour
Dep_Time_hour=[]
Dep_Time_hour=[value.split(':')[0] for value in Dep_time]

In [48]:
#selecting departure time minute
Dep_Time_min=[]
Dep_Time_min=[value.split(':')[1] for value in Dep_time]

In [49]:
#selecting Arrival time hour
Arr_Time_hour=[]
Arr_Time_hour=[value.split(':')[0] for value in Arrival_Time]

In [50]:
#selecting Arrival time min
Arr_Time_min=[]
Arr_Time_min=[value.split(':')[1] for value in Arrival_Time]

In [51]:
 Price = [price.replace(',','') for price in Price]

In [52]:
df_flights=pd.DataFrame({})
df_flights['Airline']=Airline
df_flights['Date_of_Journey']=Date_of_Journey
df_flights['Source']=Source
df_flights['Destination']=Destination
df_flights['Dep_Time_hour']=Dep_Time_hour
df_flights['Dep_Time_min']=Dep_Time_min
df_flights['Arr_Time_hour']=Arr_Time_hour
df_flights['Arr_Time_min']=Arr_Time_hour
df_flights['Duration']=Duration
df_flights['Total_Stops']=Total_Stops
df_flights['Price']=Price

In [53]:
df_flights.head(5)

,Airline,Date_of_Journey,Source,Destination,Dep_Time_hour,Dep_Time_min,Arr_Time_hour,Arr_Time_min,Duration,Total_Stops,Price
0,GO FIRST,2021-10-24,Delhi,Bangalore,19,45,22,22,02h 35m,non-stop,7424
1,SpiceJet,2021-10-24,Delhi,Bangalore,20,05,23,23,03h 05m,non-stop,7425
2,AirAsia,2021-10-24,Delhi,Bangalore,18,40,21,21,02h 50m,non-stop,7426
3,Vistara,2021-10-24,Delhi,Bangalore,20,40,23,23,02h 40m,non-stop,7425
4,Vistara,2021-10-24,Delhi,Bangalore,19,50,22,22,02h 50m,non-stop,7425


In [54]:
#Converting to the excel file
df_flights.to_excel('Flight_data1.xlsx')